In [3]:
! guardrails hub install hub://guardrails/regex_match --quiet

Installing hub://guardrails/regex_match...
✅Successfully installed guardrails/regex_match!




In [5]:
chat_history="""
nelson and murdock: i need a pickup 797 9th Avenue, manila envelope, June 3 10:00am with dropoff 10:30am Courthouse, 61 Center Street C/O frank james
operator: quote - $23.00
neslon and murdock: perfect, we accept the quote
operator: 797 9th ave, 10:00am pickup comfirmed
abc flowers: i need a pickup of a flowers from abc flowers at 21 3rd street at 11:00am on june 2 with a dropoff at 75th Ave at 5:30pm same day
operator: 21 3rd street flowers quote - $14.50
abc flowers: accepted
polk and wardell: i need a pickup of a bagels from Bakers Co at 331 5th street at 11:00am on june 3 with a dropoff at 75th Ave at 5:30pm same day
operator: 331 5th street bagels quote - $34.50
polk and wardell: accepted
"""

prompt = """
From the chat exchanges below extract a schedule of deliveries.
Chats:
${chat_history}
"""

In [3]:
from rich import print
from guardrails import Guard
from guardrails.hub import RegexMatch
from pydantic import BaseModel, Field
from typing import List

NAME_REGEX = "^[A-Z][a-z]+\\s[A-Z][a-z]+$"

class Delivery(BaseModel):
    custome_name: str= Field(validators=[RegexMatch(regex=NAME_REGEX)], description="customer name")
    pickup_time: str= Field(description="date and time of pickup")
    pickup_location: str= Field(description="address of pickup")
    dropoff_time: str= Field(description="date and time of dropoff")
    dropoff_location: str= Field(description="address of dropoff")
    price: str = Field(description="price of delivery with currency symbol included")

class Schedule(BaseModel):
    deliveries: List[Delivery]

pydantic_guard = Guard.from_pydantic(Schedule)

# Generate the function calling tool and add it to the list
pydantic_guard_tools = pydantic_guard.json_function_calling_tool([])

print(pydantic_guard_tools)

[
    {
        'type': 'function',
        'function': {
            'name': 'gd_response_tool',
            'description': 'A tool for generating responses to guardrails. It must be called last in every 
response.',
            'parameters': {
                '$defs': {
                    'Delivery': {
                        'properties': {
                            'custome_name': {
                                'description': 'customer name',
                                'title': 'Custome Name',
                                'type': 'string',
                                'validators': [{'rail_alias': 'guardrails/regex_match'}]
                            },
                            'pickup_time': {
                                'description': 'date and time of pickup',
                                'title': 'Pickup Time',
                                'type': 'string'
                            },
                            'pickup_location': {
                                'description': 'address of pickup',
                                'title': 'Pickup Location',
                                'type': 'string'
                            },
                            'dropoff_time': {
                                'description': 'date and time of dropoff',
                                'title': 'Dropoff Time',
                                'type': 'string'
                            },
                            'dropoff_location': {
                                'description': 'address of dropoff',
                                'title': 'Dropoff Location',
                                'type': 'string'
                            },
                            'price': {
                                'description': 'price of delivery with currency symbol included',
                                'title': 'Price',
                                'type': 'string'
                            }
                        },
                        'required': [
                            'custome_name',
                            'pickup_time',
                            'pickup_location',
                            'dropoff_time',
                            'dropoff_location',
                            'price'
                        ],
                        'title': 'Delivery',
                        'type': 'object'
                    }
                },
                'properties': {
                    'deliveries': {'items': {'$ref': '#/$defs/Delivery'}, 'title': 'Deliveries', 'type': 'array'}
                },
                'required': ['deliveries'],
                'type': 'object',
                'title': 'Schedule'
            },
            'required': ['deliveries']
        }
    }
]

In [4]:
rail = """
<rail version="0.1">
    <output>
        <list name="deliveries">
            <object>
                <string name="customer_name" validators="hub://guardrails/regex_match: ^[A-Z][a-z]+\\s[A-Z][a-z]+$" description="customer name" on-fail-guardrails_regex_match="reask" />
                <string name="pickup_time" description="date and time of pickup" />
                <string name="pickup_location" description="address of pickup" />
                <string name="dropoff_time" description="date and time of dropoff" />
                <string name="dropoff_location" description="address of dropoff" />
                <string name="price" description="price of delivery with currency symbol included" />
            </object>
        </list>
    </output>
</rail>
"""

rail_guard = Guard.from_rail_string(rail)


# Generate the function calling tool and add it to the list
rail_guard_tools = rail_guard.json_function_calling_tool([])

print(rail_guard_tools)

[
    {
        'type': 'function',
        'function': {
            'name': 'gd_response_tool',
            'description': 'A tool for generating responses to guardrails. It must be called last in every 
response.',
            'parameters': {
                'properties': {
                    'deliveries': {
                        'items': {
                            'properties': {
                                'customer_name': {'type': 'string', 'description': 'customer name'},
                                'pickup_time': {'type': 'string', 'description': 'date and time of pickup'},
                                'pickup_location': {'type': 'string', 'description': 'address of pickup'},
                                'dropoff_time': {'type': 'string', 'description': 'date and time of dropoff'},
                                'dropoff_location': {'type': 'string', 'description': 'address of dropoff'},
                                'price': {
                                    'type': 'string',
                                    'description': 'price of delivery with currency symbol included'
                                }
                            },
                            'required': [
                                'customer_name',
                                'pickup_time',
                                'pickup_location',
                                'dropoff_time',
                                'dropoff_location',
                                'price'
                            ],
                            'type': 'object'
                        },
                        'type': 'array'
                    }
                },
                'required': ['deliveries'],
                'type': 'object'
            },
            'required': ['deliveries']
        }
    }
]

In [19]:
import openai

response = rail_guard(
    openai.chat.completions.create,
    model="gpt-3.5-turbo",
    instructions="You are a helpful assistant.",
    prompt=prompt,
    prompt_params={"chat_history": chat_history},
    tools=rail_guard_tools,
    tool_choice="required",
)

print(response)

ValidationOutcome(
    call_id='4801405536',
    raw_llm_output='{"deliveries": [{"customer_name": "Nelson Murdock", "pickup_time": "2022-01-15T10:00:00", 
"pickup_location": "123 Main Street", "dropoff_time": "2022-01-15T12:00:00", "dropoff_location": "456 Elm Street", 
"price": "$50"}, {"customer_name": "Abc Flowers", "pickup_time": "2022-01-16T11:00:00", "pickup_location": "456 Oak
Avenue", "dropoff_time": "2022-01-16T13:00:00", "dropoff_location": "789 Pine Road", "price": "$60"}, 
{"customer_name": "Polk Wardell", "pickup_time": "2022-01-17T09:00:00", "pickup_location": "789 Maple Lane", 
"dropoff_time": "2022-01-17T11:00:00", "dropoff_location": "234 Birch Boulevard", "price": "$70"}]}',
    validated_output={
        'deliveries': [
            {
                'customer_name': 'Nelson Murdock',
                'pickup_time': 'June 3 10:00am',
                'pickup_location': '797 9th Avenue',
                'dropoff_time': 'June 3 10:30am',
                'dropoff_location': 'Courthouse, 61 Center Street C/O frank james',
                'price': '$23.00'
            },
            {
                'customer_name': 'Abc Flowers',
                'pickup_time': 'June 2 11:00am',
                'pickup_location': '21 3rd street',
                'dropoff_time': 'June 2 5:30pm',
                'dropoff_location': '75th Ave',
                'price': '$14.50'
            },
            {
                'customer_name': 'Polk Wardell',
                'pickup_time': 'June 3 11:00am',
                'pickup_location': '331 5th street',
                'dropoff_time': 'June 3 5:30pm',
                'dropoff_location': '75th Ave',
                'price': '$34.50'
            }
        ]
    },
    reask=None,
    validation_passed=True,
    error=None
)

In [21]:
rail_guard.history.last.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │                                                                                                         │ │
│   │ │ From the chat exchanges below extract a schedule of deliveries.                                         │ │
│   │ │ Chats:                                                                                                  │ │
│   │ │                                                                                                         │ │
│   │ │ nelson and murdock: i need a pickup 797 9th Avenue, manila envelope, June 3 10:00am with dropoff        │ │
│   │ │ 10:30am Courthouse, 61 Center Street C/O frank james                                                    │ │
│   │ │ operator: quote - $23.00                                                                                │ │
│   │ │ neslon and murdock: perfect, we accept the quote                                                        │ │
│   │ │ operator: 797 9th ave, 10:00am pickup comfirmed                                                         │ │
│   │ │ abc flowers: i need a pickup of a flowers from abc flowers at 21 3rd street at 11:00am on june 2 with a │ │
│   │ │ dropoff at 75th Ave at 5:30pm same day                                                                  │ │
│   │ │ operator: 21 3rd street flowers quote - $14.50                                                          │ │
│   │ │ abc flowers: accepted                                                                                   │ │
│   │ │ polk and wardell: i need a pickup of a bagels from Bakers Co at 331 5th street at 11:00am on june 3     │ │
│   │ │ with a dropoff at 75th Ave at 5:30pm same day                                                           │ │
│   │ │ operator: 331 5th street bagels quote - $34.50                                                          │ │
│   │ │ polk and wardell: accepted                                                                              │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
│   │ │ You are a helpful assistant.                                                                            │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
│   │ │ No message history.                                                                                     │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
│   │ │ {"deliveries":[{"customer_name":"nelson and murdock","pickup_time":"June 3                              │ │
│   │ │ 10:00am","pickup_location":"797 9th Avenue","dropoff_time":"June 3                                      │ │
│   │ │ 10:30am","dropoff_location":"Courthouse, 61 Center Street C/O frank                                     │ │
│   │ │ james","price":"$23.00"},{"customer_name":"abc flowers","pickup_time":"June 2                           │ │
│   │ │ 11:00am","pickup_location":"21 3rd street","dropoff_time":"June 2 5:30pm","dropoff_location":"75th      │ │
│   │ │ Ave","price":"$14.50"},{"customer_name":"po